In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
from numpy.lib.stride_tricks import as_strided
import scipy as sp

import sys
from sys import getsizeof
sys.path.append('../../')

from giagrad.nn.layers.utils import conv_output_shape
import string
from itertools import chain

In [3]:
letters = [chr(i) for i in chain(range(33, 126), range(180, 1200))]

In [33]:
## define shapes

BATCH = 1
CHANNELS = 16
INPUT_SIZE = np.array((28, 28))
KERNEL_SIZE = np.array((2, 2))
OUT_CHANNELS = 16

STRIDE = np.array((1,1))
DILATION = np.array((1,1))
PADDING = (0,) # ALWAYS ASSUME ALREADY PADDED ARRAY

INPUT_SHAPE = np.concatenate([(BATCH, CHANNELS), INPUT_SIZE])
KERNEL_SHAPE = np.concatenate([(OUT_CHANNELS, CHANNELS), KERNEL_SIZE])

OUTPUT_SIZE = conv_output_shape(
    INPUT_SHAPE, KERNEL_SIZE, STRIDE, DILATION, PADDING
)

OUTPUT_SHAPE = np.array((BATCH, OUT_CHANNELS) + tuple(OUTPUT_SIZE))

TOEPLITZ_SHAPE = np.array((np.prod(OUTPUT_SIZE)*OUT_CHANNELS, np.prod(INPUT_SIZE)*CHANNELS))

DILATED_KERNEL_SIZE = (KERNEL_SIZE - 1) * (DILATION - 1) + KERNEL_SIZE

def print_dimensions():
    print(f"{INPUT_SHAPE = }")
    print(f"{KERNEL_SHAPE = }")
    print(f"{OUTPUT_SIZE = }")
    print(f"{TOEPLITZ_SHAPE = }")
    print(f"{OUTPUT_SHAPE = }")
    print(f"{DILATED_KERNEL_SIZE = }")
    
print_dimensions()

INPUT_SHAPE = array([ 1, 16, 28, 28])
KERNEL_SHAPE = array([16, 16,  2,  2])
OUTPUT_SIZE = array([27, 27])
TOEPLITZ_SHAPE = array([11664, 12544])
OUTPUT_SHAPE = array([ 1, 16, 27, 27])
DILATED_KERNEL_SIZE = array([2, 2])


In [34]:
# init array
np.random.seed(123)
# X = np.array(letters[:np.prod(INPUT_SHAPE)], dtype=object).reshape(INPUT_SHAPE)
X = np.random.rand(*INPUT_SHAPE)
W = np.random.rand(*KERNEL_SHAPE)
T = np.zeros(TOEPLITZ_SHAPE)  # toeplitz

def print_arrays():
    print('X', X, 'W', W, 'T', T, sep='\n')
print_arrays()

X
[[[[6.96469186e-01 2.86139335e-01 2.26851454e-01 ... 3.22958914e-01
    3.61788656e-01 2.28263231e-01]
   [2.93714046e-01 6.30976124e-01 9.21049399e-02 ... 5.45068006e-01
    3.42763834e-01 3.04120789e-01]
   [4.17022211e-01 6.81300766e-01 8.75456842e-01 ... 6.91970296e-01
    5.54383250e-01 3.88950574e-01]
   ...
   [9.45236635e-01 2.90086425e-01 7.27042744e-01 ... 7.10075224e-01
    2.94516994e-01 9.71363999e-01]
   [2.78687470e-01 6.99821894e-02 5.19280361e-01 ... 6.57660241e-01
    3.24221624e-01 7.54391605e-01]
   [1.13509078e-01 7.75364763e-01 5.85901975e-01 ... 5.13679326e-01
    1.14384832e-01 5.25803385e-02]]

  [[3.30582103e-01 9.20330433e-01 9.47581835e-01 ... 6.93655394e-01
    6.89952547e-01 4.34309065e-01]
   [1.99158157e-01 9.66579376e-01 6.36908593e-02 ... 1.96346445e-01
    9.59899564e-01 4.92913728e-01]
   [7.51614963e-01 4.73991881e-01 5.87540160e-01 ... 6.72698514e-01
    6.54752644e-01 1.01746051e-01]
   ...
   [1.05683934e-01 2.78449662e-01 2.08922285e-01 ... 6.

In [35]:
# window view o Toeplitz matrix

TH, TW = T.shape
H_in, W_in = INPUT_SIZE
KH, KW = KERNEL_SIZE
H_out, W_out = OUTPUT_SIZE
stride_H, stride_W = STRIDE
dilation_H, dilation_W = DILATION
KDil_H, KDil_W = DILATED_KERNEL_SIZE

STRIDED_SHAPE = np.concatenate([OUTPUT_SIZE, (OUT_CHANNELS, CHANNELS), KERNEL_SIZE])
STRIDES = (
    TW * W_out + W_in*stride_H,
    TW + stride_W,
    TW * H_out*W_out, # out channels, jump rows in toeplitz
    H_in*W_in, # jumpt to other channel
    (W_in-KDil_W) + W_in*(dilation_H-1) + KDil_W,
    dilation_W
)

print(f"{STRIDED_SHAPE = }")
print(f"{STRIDES = }")

STRIDED_SHAPE = array([27, 27, 16, 16,  2,  2])
STRIDES = (338716, 12545, 9144576, 784, 28, 1)


In [36]:
# as_strided view
WINDOW_VIEW = as_strided(
    T, 
    STRIDED_SHAPE, 
    np.array(STRIDES)*T.itemsize)

print(f"{WINDOW_VIEW.shape = }")
print(f"{W.shape = }")

WINDOW_VIEW.shape = (27, 27, 16, 16, 2, 2)
W.shape = (16, 16, 2, 2)


In [37]:
WINDOW_VIEW += W

In [42]:
import numba as nb
from typing import *
from numpy.typing import NDArray

@nb.njit
def test(t: NDArray, x: NDArray):
    for _ in range(2**8):
        (t @ x.flatten()[:, None])

In [43]:
%%time
test(T, X)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 1d, C), Tuple(slice<a:b>, none))
 
There are 22 candidate implementations:
      - Of which 20 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(array(float64, 1d, C), Tuple(slice<a:b>, none))':
       No match.
      - Of which 2 did not match due to:
      Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 166.
        With argument(s): '(array(float64, 1d, C), Tuple(slice<a:b>, none))':
       Rejected as the implementation raised a specific error:
         NumbaTypeError: unsupported array index type none in Tuple(slice<a:b>, none)
  raised from /home/charlie/Entornos/pyIAA-GIA/lib/python3.10/site-packages/numba/core/typing/arraydecl.py:72

During: typing of intrinsic-call at /tmp/ipykernel_2

In [46]:
OUTPUT_SHAPE.dtype

dtype('int64')

In [24]:
%%time
# convolve
for _ in range(2**8):
    (T @ X.flatten()[:, None]).reshape(OUTPUT_SHAPE)

CPU times: user 1min 39s, sys: 6.04 s, total: 1min 45s
Wall time: 9.3 s


In [11]:
print_arrays()

X
[[[['!' '"' '#' '$' '%' '&']
   ["'" '(' ')' '*' '+' ',']
   ['-' '.' '/' '0' '1' '2']
   ['3' '4' '5' '6' '7' '8']
   ['9' ':' ';' '<' '=' '>']
   ['?' '@' 'A' 'B' 'C' 'D']]

  [['E' 'F' 'G' 'H' 'I' 'J']
   ['K' 'L' 'M' 'N' 'O' 'P']
   ['Q' 'R' 'S' 'T' 'U' 'V']
   ['W' 'X' 'Y' 'Z' '[' '\\']
   [']' '^' '_' '`' 'a' 'b']
   ['c' 'd' 'e' 'f' 'g' 'h']]]]
W
[[[[2 1]
   [2 2]]

  [[0 2]
   [2 1]]]


 [[[3 2]
   [3 1]]

  [[2 1]
   [0 1]]]]
T
[[2 1 0 ... 0 0 0]
 [0 2 1 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


In [12]:
T[-5:, :]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 3, 2, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 3, 2, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0,
        0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0,
        0, 0, 0, 0, 1, 0],
    